In [1]:
import spacy
from utils import tools


In [2]:
import pickle
import pandas as pd

langs_dict = {"ger":"de","fre":"fr","it":"it","en":"en","sl":"sl","pl":"pl","fi":"fi"}

with open('larger_selected_dataset.pickle', 'rb') as f:

#with open('selected_dataset.pickle', 'rb') as f:
    df = pickle.load(f)
    
df['langMaterial'] = df['langMaterial'].map(langs_dict)


In [3]:
df.head()

,id,langMaterial,unitTitle,titleProper,scopeContent,topic,filename
0,C122304194,fr,Usine Giraud,"119 J - Arnoux, fabrique de tracteurs, Miramas...",Plan des ateliers de construction V. Giraud fi...,[economics],economics.json
1,C122304196,fr,Documents généraux.,"119 J - Arnoux, fabrique de tracteurs, Miramas...",Historique par monsieur Hervé Arnoux (1993). C...,[economics],economics.json
2,C122304197,fr,Réparations et représentations pour les automo...,"119 J - Arnoux, fabrique de tracteurs, Miramas...","Garage Arnoux : vue de la façade sud, le long ...",[economics],economics.json
3,C122304198,fr,Motoculteurs Arnoux.,"119 J - Arnoux, fabrique de tracteurs, Miramas...",Brevet d'invention pour un petit tracteur moto...,[economics],economics.json
4,C122304200,fr,Tracteurs Arnoux et leur outillage.,"119 J - Arnoux, fabrique de tracteurs, Miramas...",Tracteurs type VM 10 et VM 15 : prospectus et ...,[economics],economics.json


In [4]:
topics = set([row["filename"] for index, row in df.iterrows()])

In [5]:

ngram_dict = {x:[] for x in topics}

for index, row in df.iterrows():
    lang = row["langMaterial"]
    #model = model_dict[lang]
    text = row["unitTitle"] +" "+ row["titleProper"]+" "+ row["scopeContent"]
    text = tools.strip_punct(text)
    topic = row["filename"]
    ngram_dict[topic] += tools.get_all_ngrams(text,3)


In [6]:

for t, ngrams in ngram_dict.items():
    print (t)
    selected = tools.remove_substrings(ngrams,stopwords=150,cutoff=1000)
    print (selected)
    print ( " ")

genealogy.json
['la commune', 'commune de', 'de la commune', 'la commune de', 'contenu Copies', 'en français', 'du contenu Copies', 'traduction en français', 'la Serre', 'demande de', 'de la Serre', 'Catalogue des', 'des périodiquesF204609', 'Catalogue des périodiquesF204609', 'au greffe', 'de Pradt', 'Les racines', 'racines cantaliennes', 'cantaliennes de', 'Martine Aubry', 'son père', 'JeanneJoséphine Rigal', 'la fille', 'fille de', 'plus ancien', 'la seconde', 'de Georges', 'Georges Pompidou', 'canton de', 'Les racines cantaliennes', 'racines cantaliennes de', 'la fille de', 'de Georges Pompidou', 'Denis Vieyres', 'les habitants', 'Sv Aleksandra', 'Aleksandra Ņevska', 'Ņevska baznīcas', 'Stāmerienas Sv Aleksandra', 'Sv Aleksandra Ņevska', 'Aleksandra Ņevska baznīcas', 'Ņevska baznīcas pareizticīgo', 'Rīgas cietumu', 'cietumu mācītājs', 'Rīgas cietumu mācītājs', 'Bučauskas pareizticīgo', 'Bučauskas pareizticīgo draudze', 'Rīgas Alekseja', 'Alekseja pareizticīgo', 'Rīgas Alekseja pare

In [7]:
words_dict = {}

word_langs = {}

for index, row in df.iterrows():
    lang = row["langMaterial"]
    #model = model_dict[lang]
    text = row["unitTitle"] +" "+ row["titleProper"]+" "+ row["scopeContent"]
    ngrams = get_all_ngrams(text,3)
    print (ngrams)
    break
    topic = row["filename"]
    processed_text = model(text)
    # Analyze syntax
    words = [strip_punct(chunk.text) for chunk in processed_text.noun_chunks if len(chunk.text.split(" "))<4 and len(chunk.text)>2]
    
    for entity in processed_text.ents:
        e = strip_punct(entity.text)
        if len(e)>2:
            words.append(e)
    
    for word in words:
        if word not in words_dict:
            words_dict[word] = {t:0.0 for t in topics}
            words_dict[word][topic] = 1
        else:
            words_dict[word][topic] += 1
            
        if word not in word_langs:
            word_langs[word] = {lang}
        else:
            word_langs[word].add(lang)


NameError: name 'get_all_ngrams' is not defined

In [ ]:
print (index)

In [ ]:
# this is simply for comparing across topics
ranking = {t:[] for t in topics}

for topic in topics:
    for word in words_dict.keys():
        if words_dict[word][topic]>0:
            score = words_dict[word][topic]
            ranking[topic].append([word,score])

In [ ]:
for topic,rank in ranking.items():
    print (topic)
    rank.sort(key=lambda x: x[1],reverse= True)
    for word,freq in rank[:50]:
        langs = word_langs[word]
        ents = [[Wikipedia_search(word,l),l] for l in langs]
        print (word,freq,ents)
    print (" ")

In [ ]:
# this is simply for comparing across topics
ranking = {t:[] for t in topics}

for topic in topics:
    for word in words_dict.keys():
        if words_dict[word][topic]>0:
            mean = [y for x,y in words_dict[word].items() if x!=topic]
            mean = sum(mean)/len(mean)
            if mean != 0:
                score = words_dict[word][topic] - mean
                ranking[topic].append([word,score])

In [ ]:
for topic,rank in ranking.items():
    print (topic)
    rank.sort(key=lambda x: x[1],reverse= True)
    for el in rank[:50]:
        print (el)
    print (" ")